<a href="https://colab.research.google.com/github/DanhChun/Log-Analyzer-with-NLP-model/blob/main/Distilation_logs_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LOAD DỮ LIỆU VÀ KIỂM TRA DỮ LIỆU**

In [ ]:
from google.colab import files
import pandas as pd

# Tải lên file từ máy tính
uploaded = files.upload()



Saving labeled_logs.csv to labeled_logs.csv


In [ ]:
# Đọc dữ liệu từ file CSV

df = pd.read_csv("labeled_logs.csv")

# Hiển thị một vài dòng đầu tiên
df.head()

,logs,labels
0,00:00:13.122..\src\mgw3\mgwp.py.282.INFO: -> d...,Normal
1,00:00:13.123..\src\mgw3\mgwp.py.50.INFO: -> ap...,Normal
2,00:00:13.124..\src\mgw3\mgwp.py.349.INFO:app_c...,NOTE - all threads closed
3,00:00:13.125..\src\mgw3\sif\sif_mgr.py.68.INFO...,Normal
4,00:00:13.125..\src\mgw3\sif\sif_mgr.py.70.INFO...,Normal


**LÀM SẠCH DỮ LIỆU**

In [ ]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv("labeled_logs.csv")

# Chuyển logs thành chữ thường (uncased)
df["logs"] = df["logs"].str.lower()

# Giữ nguyên labels không thay đổi
df["labels"] = df["labels"].str.strip()  # Loại bỏ khoảng trắng thừa nếu có

df.dropna(inplace=True)  # Xóa dòng có giá trị rỗng

# Hiển thị một số dòng đầu của dữ liệu
print(df.head())


                                                logs  \
0  00:00:13.122..\src\mgw3\mgwp.py.282.info: -> d...   
1  00:00:13.123..\src\mgw3\mgwp.py.50.info: -> ap...   
2  00:00:13.124..\src\mgw3\mgwp.py.349.info:app_c...   
3  00:00:13.125..\src\mgw3\sif\sif_mgr.py.68.info...   
4  00:00:13.125..\src\mgw3\sif\sif_mgr.py.70.info...   

                      labels  
0                     Normal  
1                     Normal  
2  NOTE - all threads closed  
3                     Normal  
4                     Normal  


**TẠO WORD MAP CHO CÁC BIẾN THỂ DỮ LIỆU**

In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pandas as pd
import random

# Đọc dữ liệu logs
df = pd.read_csv("labeled_logs.csv")

#  Với lỗi chứa từ khóa có thể thay thế (thay thế từ đồng nghĩa)
synonym_map = {
    "problem": ["issue", "fault", "malfunction"],
    "error": ["failure", "fault", "bug"],
    "failed": ["crashed", "unsuccessful", "did not succeed"],
    "download": ["get", "acquire", "http_download", "missing","fetch data", "data retrieval"],
    "disconnect": ["lost connection", "disconnection", "drop"],
    "checksum": ["CRC", "hash check", "data integrity check"],
    "message size": ["packet size", "data length", "buffer size"]
}

#  Với lỗi chứa thông số kỹ thuật hoặc tên thiết bị
device_map = {
    "control": ["amplifier control", "control_amplifier", "control_ampli", "control amplifier"],
    "SIM": ["subscriber identity module", "mobile SIM", "network SIM"],
    "GSM": ["cellular network", "mobile network", "telecom system"],
}

#  Với các thông báo mang tính mô tả
phrase_map = {
    "all threads closed": ["all processes terminated", "all worker threads stopped"],
    "message played done": ["audio playback finished", "sound message completed", "-> playing changed: ON -> OFF", "ON -> OFF", "playing changed"],
    "speaker changes state form Off to On": ["turn LED RUN", "LED RUN", "turn on speaker"],
    "speaker changes state form On to Off": ["turn LED OFF", "LED OFF", "LED -> OFF"]
}

#  Với nhãn đặc biệt như "Normal"
normal_variations = [
    "System stable",
    "No issues detected",
    "Operating normally",
    "Everything is running smoothly"
]

# Hàm thay thế từ đồng nghĩa và thuật ngữ liên quan
def augment_text(text):
    original_text = text  # Lưu văn bản gốc để so sánh sau

    # Thay thế từ đồng nghĩa
    for phrase, synonyms in synonym_map.items():
        if phrase in text:
            synonym = random.choice(synonyms)
            text = text.replace(phrase, synonym)

    # Thay thế thuật ngữ thiết bị
    for phrase, variations in device_map.items():
        if phrase in text:
            variation = random.choice(variations)
            text = text.replace(phrase, variation)

    # Thay đổi cách diễn đạt của mô tả lỗi
    for phrase, variations in phrase_map.items():
        if phrase in text:
            variation = random.choice(variations)
            text = text.replace(phrase, variation)

    # Nếu nhãn là "Normal", thay thế ngẫu nhiên bằng cách diễn đạt khác
    if text.strip().lower() == "normal":
        text = random.choice(normal_variations)

    # Kiểm tra nếu có thay đổi
    if original_text != text:
        print(f"Original: {original_text}")
        print(f"Augmented: {text}")

    return text

# Áp dụng augmentation cho toàn bộ tập dữ liệu
df["augmented_logs"] = df["logs"].apply(augment_text)

# Kiểm tra dữ liệu đã augmented
print(df.head())  # In ra vài dòng đầu tiên của dữ liệu đã thay đổi

# Lưu kết quả
df.to_csv("augmented_labeled_logs.csv", index=False)

Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli failure
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli bug
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli fault
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control amplifier_ampli fault
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli_ampli failure
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\httt

In [ ]:
# Kiểm tra nếu có sự thay đổi giữa logs và augmented_logs
for i, row in df.iterrows():
    if row['logs'] != row['augmented_logs']:
        print(f"Original: {row['logs']}")
        print(f"Augmented: {row['augmented_logs']}")
        print('-' * 50)


Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli failure
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: amplifier control_ampli bug
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_amplifier_ampli fault
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control amplifier_ampli fault
--------------------------------------------------
Original: 00:00:46.938..\src\mgw3\htttn\htttn_rx.py.93.ERROR: control_ampli error
Augmented: 00:

**TOKEN HOÁ VÀ MAPPING NHÃN**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('augmented_labeled_logs.csv')

# Mã hóa nhãn thành số
label_encoder = LabelEncoder()
df['encoded_labels'] = label_encoder.fit_transform(df['labels'])

# Lưu lại dữ liệu với nhãn đã mã hóa
df.to_pickle('encoded_labeled_logs.pkl')  # Lưu dưới dạng pickle
df.to_csv('encoded_labeled_logs.csv', index=False)  # Lưu dưới dạng CSV

# Xem trước một số mẫu nhãn đã mã hóa
print(df[['labels', 'encoded_labels']].head())


                      labels  encoded_labels
0                     Normal              11
1                     Normal              11
2  NOTE - all threads closed               8
3                     Normal              11
4                     Normal              11


In [ ]:
# Liệt kê các nhãn gốc và nhãn mã hóa tương ứng
encoded_labels_mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))
print("Nhãn gốc và nhãn mã hóa:")
for label, encoded in encoded_labels_mapping.items():
    print(f"{label}: {encoded}")


Nhãn gốc và nhãn mã hóa:
ERROR - MPD problem: 0
ERROR - config SIM, GSM: 1
ERROR - control amplifier: 2
ERROR - delete message: 3
ERROR - invalid checksum: 4
ERROR - invalid message size: 5
ERROR - message download: 6
ERROR - start MPD failed: 7
NOTE - all threads closed: 8
NOTE - speaker changes state from Off to On: 9
NOTE - speaker changes state from On to Off: 10
Normal: 11
Note - message played done: 12
WARNING - On disconnect: 13


In [ ]:
df = df.dropna()


In [ ]:
import torch
from transformers import DistilBertTokenizer

# Đọc dữ liệu đã mã hóa nhãn từ tệp CSV
df = pd.read_csv('encoded_labeled_logs.csv')

# Tải tokenizer của DistilBERT từ Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Chuyển các cột pandas Series thành list
train_texts = df['augmented_logs'][:int(0.8 * len(df))].astype(str).tolist()
val_texts = df['augmented_logs'][int(0.8 * len(df)):].astype(str).tolist()

# Tokenize train và validation set
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# Chuyển đổi các encoding thành tensor
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
val_input_ids = torch.tensor(val_encodings['input_ids'])
val_attention_mask = torch.tensor(val_encodings['attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# In ra 5 câu đã được tokenized và input ids tương ứng
print("Sample tokenized text:")
for i in range(5):
    print(f"Original text: {train_texts[i]}")
    print(f"Tokenized ids: {train_encodings['input_ids'][i]}")
    print(f"Attention mask: {train_encodings['attention_mask'][i]}")
    print("-" * 40)


Sample tokenized text:
Original text: 00:00:13.122..\src\mgw3\mgwp.py.282.INFO: -> do RESTART APP at night!
Tokenized ids: [101, 4002, 1024, 4002, 1024, 2410, 1012, 13092, 1012, 1012, 1032, 5034, 2278, 1032, 11460, 2860, 2509, 1032, 11460, 2860, 2361, 1012, 1052, 2100, 1012, 26267, 1012, 18558, 1024, 1011, 1028, 2079, 23818, 10439, 2012, 2305, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
inputs = tokenizer("Error: http_download error", return_tensors="pt", truncation=True, padding=True, max_length=128)
print("Input IDs:", inputs["input_ids"])
print("Attention Mask:", inputs["attention_mask"])


Input IDs: tensor([[ 101, 7561, 1024, 8299, 1035, 8816, 7561,  102]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
print(tokenizer.convert_ids_to_tokens(1035))


_


**TẠO DATASET**

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Mã hóa nhãn thành tensor
train_labels = torch.tensor(df['encoded_labels'][:int(0.8 * len(df))].tolist())
val_labels = torch.tensor(df['encoded_labels'][int(0.8 * len(df)):].tolist())

# Tạo Dataset cho PyTorch
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Tạo dataset cho train và validation
train_dataset = CustomDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = CustomDataset(val_input_ids, val_attention_mask, val_labels)

# Tạo DataLoader cho train và validation
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16)

# Lưu lại tệp dữ liệu tokenized
df['input_ids'] = list(train_input_ids.numpy()) + list(val_input_ids.numpy())
df['attention_mask'] = list(train_attention_mask.numpy()) + list(val_attention_mask.numpy())
df['encoded_labels'] = list(train_labels.numpy()) + list(val_labels.numpy())

# Lưu dưới dạng pickle và CSV
df.to_pickle('tokenized_encoded_labeled_logs.pkl')  # Lưu dưới dạng pickle
df.to_csv('tokenized_encoded_labeled_logs.csv', index=False)  # Lưu dưới dạng CSV


**CẤU HÌNH TRAIN**

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

num_labels = len(set(df["labels"]))  # Số lượng nhãn

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**CẤU HÌNH HÀM ĐÁNH GIÁ MÔ HÌNH**

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Hàm tính độ chính xác
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Chọn nhãn có xác suất cao nhất
    acc = accuracy_score(labels, predictions)  # Tính độ chính xác
    return {"accuracy": acc}

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Chọn nhãn có xác suất cao nhất
    acc = accuracy_score(labels, predictions)  # Tính độ chính xác
    precision = precision_score(labels, predictions, average='weighted')  # Tính precision
    recall = recall_score(labels, predictions, average='weighted')  # Tính recall
    f1 = f1_score(labels, predictions, average='weighted')  # Tính F1-score
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

**THIẾT LẬP THÔNG SỐ TRAIN**

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",  # Tắt W&B
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # 🔹 Đảm bảo số epochs như mong muốn
    weight_decay=0.01,
    logging_dir="./logs",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch

# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # Kiểm tra thiết bị đang sử dụng


Using device: cuda


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


**TIẾN HÀNH TRAIN**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
from sklearn.metrics import balanced_accuracy_score
import numpy as np

# Hàm tính Balanced Accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "balanced_accuracy": balanced_accuracy_score(labels, preds)
    }

# Lớp Trainer chuẩn sử dụng Cross Entropy Loss
class SimpleTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # Lấy labels
        outputs = model(**inputs)  # Truyền inputs vào model
        logits = outputs.logits  # Lấy logits

        # Sử dụng Cross Entropy Loss mà không có class weights
        loss_fct = nn.CrossEntropyLoss()  # Không có trọng số lớp
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Khởi tạo Trainer với Cross Entropy Loss và balanced accuracy
trainer = SimpleTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Balanced Accuracy
1,No log,0.814355,0.906940
2,No log,0.230708,0.997900
3,0.324200,0.144020,0.997900


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


TrainOutput(global_step=723, training_loss=0.23493404111426897, metrics={'train_runtime': 148.6567, 'train_samples_per_second': 77.817, 'train_steps_per_second': 4.864, 'total_flos': 383177700532224.0, 'train_loss': 0.23493404111426897, 'epoch': 3.0})

**ĐÁNH GIÁ KẾT QUẢ MÔ HÌNH**

In [ ]:
trainer.evaluate()


{'eval_loss': 0.14402028918266296,
 'eval_balanced_accuracy': 0.9979002264461675,
 'eval_runtime': 3.197,
 'eval_samples_per_second': 301.533,
 'eval_steps_per_second': 19.08,
 'epoch': 3.0}

In [ ]:
model.save_pretrained("./distilbert_log_classifier")
tokenizer.save_pretrained("./distilbert_log_classifier")


('./distilbert_log_classifier/tokenizer_config.json',
 './distilbert_log_classifier/special_tokens_map.json',
 './distilbert_log_classifier/vocab.txt',
 './distilbert_log_classifier/added_tokens.json')

**KIỂM TRA HOẠT ĐỘNG CỦA MÔ HÌNH MỚI**

In [ ]:
import torch

# Lấy thiết bị của mô hình (CPU hoặc CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Đưa mô hình về đúng thiết bị

def predict_log(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Chuyển input lên cùng thiết bị với model

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# Ví dụ dự đoán
log_sample = "Error: invalid checksum"
print(predict_log(log_sample))


ERROR - invalid checksum


In [ ]:
def predict_log_proba(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    label = label_encoder.inverse_transform([predicted_class])[0]
    confidence = probabilities[0][predicted_class].item()

    return label, confidence

# Ví dụ dự đoán
log_sample = "16:46:11.98..\src\mgw3\mylib\io.py.92.INFO:config SIM, GSM error"
predicted_label, confidence = predict_log_proba(log_sample)
print(f"Predicted: {predicted_label} (Confidence: {confidence:.2f})")


Predicted: ERROR - config SIM, GSM (Confidence: 0.94)


In [ ]:
def predict_log_verbose(log_text):
    inputs = tokenizer(log_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)  # Chuyển sang xác suất
    predicted_class = torch.argmax(logits, dim=1).item()

    print("Logits:", logits)
    print("Probabilities:", probabilities)
    print("Predicted class index:", predicted_class)
    print("Predicted label:", label_encoder.inverse_transform([predicted_class])[0])

    return label_encoder.inverse_transform([predicted_class])[0]

# Chạy thử
log_sample = "13:40:57.691..\src\mgw3\htttn\htttn_mgr.py.357.INFO:_on_disconnect: client=<paho.mqtt.client.Client object at 0xb57c4c58>, rc=0"
predict_log_verbose(log_sample)


Logits: tensor([[-1.5782, -0.7780, -0.8959, -0.6367, -1.1741, -1.2046, -0.5707, -0.7973,
         -1.0537, -0.8489, -0.3020, -0.0745, -1.0807,  5.3687]],
       device='cuda:0')
Probabilities: tensor([[9.3540e-04, 2.0822e-03, 1.8506e-03, 2.3981e-03, 1.4011e-03, 1.3591e-03,
         2.5618e-03, 2.0423e-03, 1.5804e-03, 1.9397e-03, 3.3516e-03, 4.2076e-03,
         1.5383e-03, 9.7275e-01]], device='cuda:0')
Predicted class index: 13
Predicted label: WARNING - On disconnect


'WARNING - On disconnect'

**LƯU MÔ HÌNH DƯỚI DẠNG pt**

In [ ]:
import torch

# Lưu mô hình
torch.save(model.state_dict(), "distilbert_log_classifier.pt")

# Khi cần tải lại
model.load_state_dict(torch.load("distilbert_log_classifier.pt"))
model.eval()  # Đặt chế độ đánh giá (không huấn luyện lại)



DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


**KIỂM TRA KÍCH THƯỚC MÔ HÌNH**

In [ ]:
import os

file_path = "distilbert_log_classifier.pt"
file_size = os.path.getsize(file_path) / (1024 * 1024)  # Chuyển sang MB
print(f"Kích thước mô hình: {file_size:.2f} MB")


Kích thước mô hình: 255.49 MB


**LƯU MÔ HÌNH DƯỚI DẠNG TF**

In [ ]:
import tensorflow as tf

# Gọi lại mô hình được huấn luyện
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(20,)),
    tf.keras.layers.Dense(5, activation='softmax')
])

# Lưu mô hình dưới dạng SavedModel
model.save("saved_model_dir")


**CHUYỂN ĐỔI SANG TF LITE**

In [ ]:
import tensorflow as tf

# Tạo converter từ SavedModel đã lưu
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_dir")

# (Tùy chọn) Tối ưu hóa mô hình
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# (Tùy chọn)  float16
#converter.target_spec.supported_types = [tf.float16]

# Convert sang mô hình TFLite
tflite_model = converter.convert()

# Ghi mô hình TFLite ra file
with open("logs_analyse_model.tflite", "wb") as f:
    f.write(tflite_model)
